In [109]:
using Random
using DataFrames
using CSV
using Statistics
using Printf

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 100
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int, f::Function = random_fighter)::Fighter
    if c==0
        return f()
    end
    a = random_tournament_winner(c-1, f)
    b = random_tournament_winner(c-1, f)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [59]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [60]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [61]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [62]:
function evaluate_generator2(f1, ts, np, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    t_a = random_team(f1, ts)
    f_b = pick_best_rdmly(library, t_a, np)
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            t_a = random_team(f1, ts)
            f_b = pick_best_rdmly(library, t_a, np)        
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

evaluate_generator2 (generic function with 2 methods)

In [86]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    name = ""
    while budget_sofar > budget
        hf_ratio = rand() * .75 + 0.02
        l_ratio = rand()
        h_temp = rand() + 0.07
        f_temp = rand() + 0.07
        name = string("Scheme-A (", @sprintf("%6.3f",hf_ratio), ",", @sprintf("%6.3f",l_ratio), ",", @sprintf("%6.3f",h_temp), ",", @sprintf("%6.3f",f_temp), ")")
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.07
                f_temp = rand() + 0.07
            end
        end
        budget_hf = (0.07 + hf_ratio * 0.8) * (budget * .75)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter(name, h, f, l, t)
end

@time evaluate_generator2(random_scheme_a, 100, 100, 50000)

  8.145173 seconds (45.40 M allocations: 7.848 GiB, 14.28% gc time)


(0.66578, 1.0)

In [72]:
random_scheme_a()

Fighter("Scheme-A ( 0.698, 0.926, 0.732, 1.011)", 146, 32, 33, 15)

In [56]:
@time evaluate_generator(random_scheme_a, 2000, 5)

  9.189599 seconds (2.00 M allocations: 45.854 MiB, 0.07% gc time)


-0.3412

In [73]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = (h-2)^2/((h-2)^2+(f-2)^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(60.0, 83.0, 31.0, 38.0, 4980.0, 9934.0, 0.7747356505626481)


## Testing Random Scheme A

In [74]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("Scheme-A ( 0.135, 0.846, 0.500, 0.364)", 49, 27, 134, 78)

9995

In [81]:
rt = random_team(() -> random_scheme_a(), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,"Scheme-A ( 0.105, 0.536, 0.288, 0.638)",23,51,71,141
2,"Scheme-A ( 0.429, 0.761, 0.138, 0.943)",21,146,17,45
3,"Scheme-A ( 0.061, 0.932, 0.795, 0.746)",49,18,162,65
4,"Scheme-A ( 0.101, 0.317, 0.868, 0.673)",93,12,92,27
5,"Scheme-A ( 0.191, 0.346, 0.363, 0.917)",26,65,49,108
6,"Scheme-A ( 0.728, 0.764, 0.372, 0.918)",45,110,21,37
7,"Scheme-A ( 0.129, 0.481, 0.649, 0.363)",48,27,135,82
8,"Scheme-A ( 0.729, 0.048, 0.266, 0.659)",44,110,22,38
9,"Scheme-A ( 0.180, 0.901, 0.961, 0.769)",64,25,112,49


In [82]:
counters = random_team(() -> pick_best_rdmly(library, rt, 99), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Jara the Scissors,42,52,84,82
2,Ostrich Zyahir,51,47,77,78
3,Cheese Silah,41,45,100,90
4,Enthusiastic Wombat,43,56,80,74
5,Energetic Khaison,44,44,95,88
6,Koren the Zipper,43,45,87,96
7,Dyon the Soldier,40,56,68,90
8,Bagel Taylah,46,51,93,66
9,Tag the Penguin,39,58,98,67


In [83]:
rt = random_team(() -> random_scheme_a(), 100)

100-element Vector{Fighter}:
 Fighter("Scheme-A ( 0.475, 0.794, 1.035, 0.195)", 134, 25, 46, 19)
 Fighter("Scheme-A ( 0.050, 0.679, 0.107, 0.615)", 12, 69, 34, 127)
 Fighter("Scheme-A ( 0.270, 0.214, 0.224, 0.873)", 23, 91, 27, 80)
 Fighter("Scheme-A ( 0.530, 0.853, 0.670, 0.510)", 23, 163, 16, 36)
 Fighter("Scheme-A ( 0.135, 0.633, 0.367, 1.033)", 22, 61, 52, 123)
 Fighter("Scheme-A ( 0.656, 0.635, 0.443, 0.359)", 18, 244, 13, 22)
 Fighter("Scheme-A ( 0.668, 0.535, 0.292, 1.059)", 35, 128, 18, 38)
 Fighter("Scheme-A ( 0.189, 0.563, 0.074, 0.627)", 14, 119, 17, 68)
 Fighter("Scheme-A ( 0.114, 0.250, 0.416, 0.305)", 23, 52, 68, 139)
 Fighter("Scheme-A ( 0.366, 0.874, 0.513, 0.842)", 41, 67, 51, 77)
 Fighter("Scheme-A ( 0.651, 0.087, 0.268, 0.080)", 86, 52, 45, 31)
 Fighter("Scheme-A ( 0.586, 0.262, 0.603, 0.601)", 35, 116, 20, 45)
 Fighter("Scheme-A ( 0.645, 0.767, 0.756, 0.251)", 115, 38, 42, 21)
 ⋮
 Fighter("Scheme-A ( 0.218, 0.392, 0.683, 0.905)", 26, 71, 43, 99)
 Fighter("Scheme-A (

In [87]:
for f in rt[1:20]
    v = eval_battle_list(f, counters)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Scheme-A ( 0.475, 0.794, 1.035, 0.195)", 134, 25, 46, 19)
-100
Fighter("Scheme-A ( 0.270, 0.214, 0.224, 0.873)", 23, 91, 27, 80)
-40
Fighter("Scheme-A ( 0.530, 0.853, 0.670, 0.510)", 23, 163, 16, 36)
-92
Fighter("Scheme-A ( 0.656, 0.635, 0.443, 0.359)", 18, 244, 13, 22)
-92
Fighter("Scheme-A ( 0.668, 0.535, 0.292, 1.059)", 35, 128, 18, 38)
-92
Fighter("Scheme-A ( 0.189, 0.563, 0.074, 0.627)", 14, 119, 17, 68)
-66
Fighter("Scheme-A ( 0.114, 0.250, 0.416, 0.305)", 23, 52, 68, 139)
78
Fighter("Scheme-A ( 0.366, 0.874, 0.513, 0.842)", 41, 67, 51, 77)
-34
Fighter("Scheme-A ( 0.651, 0.087, 0.268, 0.080)", 86, 52, 45, 31)
78
Fighter("Scheme-A ( 0.586, 0.262, 0.603, 0.601)", 35, 116, 20, 45)
-92
Fighter("Scheme-A ( 0.645, 0.767, 0.756, 0.251)", 115, 38, 42, 21)
-84
Fighter("Scheme-A ( 0.614, 0.033, 0.210, 0.095)", 25, 168, 16, 32)
-92
Fighter("Scheme-A ( 0.633, 0.494, 0.339, 0.703)", 46, 95, 27, 46)
-42
Fighter("Scheme-A ( 0.506, 0.569, 0.439, 0.704)", 48, 75, 41, 59)
-22
Fighter("Sch

In [88]:
for f in counters[1:20]
    v = eval_battle_list(f, rt)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Jara the Scissors", 42, 52, 84, 82)
46
Fighter("Ostrich Zyahir", 51, 47, 77, 78)
38
Fighter("Cheese Silah", 41, 45, 100, 90)
46
Fighter("Enthusiastic Wombat", 43, 56, 80, 74)
42
Fighter("Energetic Khaison", 44, 44, 95, 88)
48
Fighter("Koren the Zipper", 43, 45, 87, 96)
48
Fighter("Dyon the Soldier", 40, 56, 68, 90)
46
Fighter("Bagel Taylah", 46, 51, 93, 66)
32
Fighter("Tag the Penguin", 39, 58, 98, 67)
38
Fighter("Joven the Giant", 45, 55, 90, 63)
28
Fighter("Candle Kamaro", 44, 56, 87, 66)
40
Fighter("Powerful Jellyfish", 39, 59, 96, 67)
38
Fighter("Matheus the Tiger", 38, 52, 82, 94)
36
Fighter("Powerful Train", 44, 46, 94, 83)
48
Fighter("Ahmoni the Noble", 42, 50, 96, 77)
42
Fighter("Wombat Akan", 42, 44, 89, 100)
50
Fighter("Sheltering Xerox", 60, 35, 75, 97)
30
Fighter("Martini Judsen", 39, 51, 91, 87)
44
Fighter("Ahavah the Koala", 42, 49, 81, 92)
46
Fighter("Lema the Eel", 50, 48, 67, 88)
42


In [89]:
for f in rt[1:20]
    v = eval_battle_list(f, counters)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Scheme-A ( 0.050, 0.679, 0.107, 0.615)", 12, 69, 34, 127)
92
Fighter("Scheme-A ( 0.135, 0.633, 0.367, 1.033)", 22, 61, 52, 123)
92
Fighter("Scheme-A ( 0.114, 0.250, 0.416, 0.305)", 23, 52, 68, 139)
78
Fighter("Scheme-A ( 0.651, 0.087, 0.268, 0.080)", 86, 52, 45, 31)
78


In [90]:
evaluate_generator(random_scheme_a, 100, 20)

-0.4590000000000001

In [91]:
evaluate_generator(random_fighter, 100, 20)

-0.8535

In [92]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.8355

In [93]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.8360000000000001

In [94]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.8290000000000001

In [95]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.799

## Team battles

In [110]:
random_tournament_winner(2, random_scheme_a)

Fighter("Scheme-A ( 0.340, 0.083, 0.110, 0.175)", 64, 40, 81, 56)

In [111]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_a1 = random_team(() -> rand_rename(random_tournament_winner(1, random_scheme_a)), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Triangle Hari", 43, 96, 20, 52)
 Fighter("Mickayla the Tyro", 78, 36, 67, 54)
 Fighter("Tribe the Hunter", 24, 94, 99, 57)
 Fighter("Haiden the Responsible", 21, 84, 76, 79)
 Fighter("Strong Soldier", 44, 66, 30, 87)
 Fighter("Monstrous Zacharias", 42, 94, 43, 45)
 Fighter("Staley the Robot", 44, 81, 48, 53)
 Fighter("Yellow Heron", 77, 63, 20, 57)
 Fighter("Kathryne the Shy", 64, 88, 36, 23)
 Fighter("Xray Nairah", 38, 63, 47, 92)
 Fighter("Majesti the Notebook", 74, 72, 30, 34)
 Fighter("Risky Baboon", 31, 81, 22, 84)
 Fighter("Icicle Tel", 16, 85, 88, 85)
 ⋮
 Fighter("Yashi the Merciful", 60, 72, 82, 10)
 Fighter("Ingenious Balloon", 60, 56, 95, 16)
 Fighter("Car Penelopi", 40, 52, 68, 100)
 Fighter("Rahsaan the Blind", 74, 29, 72, 87)
 Fighter("Anaia the Warm", 16, 97, 91, 72)
 Fighter("Demarcus the Minimal", 56, 100, 39, 22)
 Fighter("Jellyfish Iyonna", 48, 85, 86, 21)
 Fighter("Kriyaan the Bear", 43, 75, 26, 75)
 Fighter("Jayani the Cricket"

In [112]:
eval_team_battle(team_a, counters)

1

In [113]:
eval_team_battle(team_a, team_r0)

1

In [114]:
eval_team_battle(team_a, team_r1)

1

In [115]:
eval_team_battle(team_a, team_r2)

1

In [116]:
eval_team_battle(team_a, team_a1)

1

In [101]:
eval_team_battle(team_r0, team_a)

-1

In [102]:
eval_team_battle(team_r0, team_r1)

-1

In [103]:
eval_team_battle(team_r0, team_r2)

-1

In [117]:
as = team_a
bs = team_a1
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Sassy Mango", 82, 32, 76, 35)
         Fighter("Soldier Finley", 32, 85, 33, 73)
Sassy Mango lost against Soldier Finley
         Fighter("Generous Student", 39, 18, 194, 96)
Generous Student lost against Soldier Finley
         Fighter("Satsuki the Blanket", 129, 30, 43, 19)
Satsuki the Blanket won against Soldier Finley
         Fighter("Triangle Hadli", 44, 70, 47, 69)
Satsuki the Blanket lost against Triangle Hadli
         Fighter("Kalliope the Valiant", 29, 53, 72, 120)
Kalliope the Valiant won against Triangle Hadli
         Fighter("Blessed the Robot", 26, 113, 23, 57)
Kalliope the Valiant won against Blessed the Robot
         Fighter("Underwater Philosopher", 127, 17, 59, 20)
Kalliope the Valiant won against Underwater Philosopher
         Fighter("Mouse Elenoa", 114, 41, 39, 21)
Kalliope the Valiant lost against Mouse Elenoa
         Fighter("Young Roselle", 103, 44, 42, 25)
Young Roselle won against Mouse Elenoa
         Fighter("Nitya the Kingly", 24, 136

Tangled Marksman lost against Lexani the Pug
         Fighter("Luminous Shark", 23, 61, 52, 121)
Luminous Shark won against Lexani the Pug
         Fighter("Jhett the Hunter", 41, 90, 30, 56)
Luminous Shark won against Jhett the Hunter
         Fighter("Commander Kessiah", 69, 40, 77, 48)
Luminous Shark lost against Commander Kessiah
         Fighter("Zen the Blueberry", 87, 42, 57, 33)
Zen the Blueberry lost against Commander Kessiah
         Fighter("Xpanded Lawyer", 39, 67, 51, 80)
Xpanded Lawyer lost against Commander Kessiah
         Fighter("Giraffe Jamesyn", 34, 95, 28, 61)
Giraffe Jamesyn won against Commander Kessiah
         Fighter("Dynamic Duck", 84, 51, 48, 33)
Giraffe Jamesyn won against Dynamic Duck
         Fighter("Warrior Norvin", 26, 69, 44, 102)
Giraffe Jamesyn lost against Warrior Norvin
         Fighter("Aiven the Puppy", 43, 18, 180, 82)
Aiven the Puppy lost against Warrior Norvin
         Fighter("Victorious Engine", 25, 38, 112, 164)
Victorious Engine won again

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [118]:
a_i, b_i

(93, 101)